In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from os import listdir
train_data_li = listdir('./train/')
train_data_li

import sys
mod = sys.modules[__name__]
for name in train_data_li:
    setattr(mod, name.split('.')[0], pd.read_csv('./train/' + name))
    
train_activity.shape, train_combat.shape, train_pledge.shape, train_trade.shape, train_payment.shape, train_label.shape

((1601834, 17),
 (1574671, 13),
 (1083384, 15),
 (1849325, 11),
 (66432, 3),
 (40000, 3))

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Masking
from keras.optimizers import RMSprop
from keras import backend as k
from sklearn.preprocessing import normalize

def weibull_loglik_discrete(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    hazard0 = k.pow((y_ + 1e-35) / a_, b_)
    hazard1 = k.pow((y_ + 1) / a_, b_)

    return -1 * k.mean(u_ * k.log(k.exp(hazard1 - hazard0) - 1.0) - hazard1)

def weibull_loglik_continuous(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    ya = (y_ + 1e-35) / a_
    return -1 * k.mean(u_ * (k.log(b_) + b_ * k.log(ya)) - k.pow(ya, b_))

def activate(ab):
    a = k.exp(ab[:, 0])
    b = k.softplus(ab[:, 1])

    a = k.reshape(a, (k.shape(a)[0], 1))
    b = k.reshape(b, (k.shape(b)[0], 1))

    return k.concatenate((a, b), axis=1)

Using TensorFlow backend.


# 진영이형의 준비된 시계열 데이터로 WTTE 학습

In [5]:
%%time
activity = train_activity.groupby(['acc_id','day']).sum().reset_index().drop('char_id',axis=1)
combat = train_combat.groupby(['acc_id','day']).sum().reset_index().drop('char_id',axis=1)
payment = train_payment.groupby(['acc_id','day']).sum().reset_index()
pledge = train_pledge.groupby(['acc_id','day']).sum().reset_index().drop('char_id',axis=1)

Wall time: 7 s


In [7]:
%%time
# 주는 유저
train_trade_source = train_trade[['day', 'time', 'type', 'server', 'source_acc_id', 
                            'source_char_id', 'item_type', 'item_amount','item_price']]
# 받는 유저
train_trade_target = train_trade[['day', 'time', 'type', 'server', 'target_acc_id', 
                            'target_char_id', 'item_type', 'item_amount','item_price']]

train_trade_source = train_trade_source.rename(columns = {
                                            'source_acc_id': 'acc_id', 
                                            'source_char_id':'char_id',
                                            'item_amount': 'item_amount_s', 
                                            'item_price': 'item_price_s'})
train_trade_target = train_trade_target.rename(columns = {
                                            'target_acc_id': 'acc_id', 
                                            'target_char_id':'char_id',
                                            'item_amount': 'item_amount_t', 
                                            'item_price': 'item_price_t'})

Wall time: 1.04 s


In [10]:
train_trade_target.groupby(['day', 'acc_id', 'char_id']).sum().reset_index()

,day,acc_id,char_id,type,item_amount_t,item_price_t
0,1,17,135044,1,7.191000e-02,0.011379
1,1,20,149667,1,2.396984e-08,0.000000
2,1,22,224321,1,2.924337e-01,0.009878
3,1,33,966,0,2.396984e-08,0.001019
4,1,53,60446,1,7.192606e-03,0.036819
5,1,62,369719,4,1.078645e+00,0.004447
6,1,65,270517,3,1.271329e+00,0.000000
7,1,79,225036,8,1.246416e+00,0.000000
8,1,79,331572,5,5.666636e-01,0.000000
9,1,99,232152,0,4.793968e-08,0.157812


In [16]:
%%time
# '시간' 컬럼을 그룹별로 합침(time, item_type, server 제거됨  )
train_trade_source = train_trade_source.groupby(['day','acc_id','char_id']).sum()
train_trade_source = train_trade_source.reset_index()

train_trade_target = train_trade_target.groupby(['day','acc_id','char_id']).sum()
train_trade_target = train_trade_target.reset_index()

train_trade_target['item_price_t'] = train_trade_target['item_price_t']*(-1)

standard = activity[['acc_id','day','playtime']]
source = pd.merge(standard, train_trade_source, 
                  on=['day','acc_id'], how = 'outer').dropna(
            subset = ['playtime']).drop(['char_id','playtime'],axis=1)
target = pd.merge(standard, train_trade_target, 
                  on=['day','acc_id'], how = 'outer').dropna(
            subset = ['playtime']).drop(['char_id','playtime'],axis=1)

trade = pd.merge(source, target , on=['day','acc_id'], how = 'outer')
trade = trade.groupby(['acc_id','day']).sum().reset_index()

Wall time: 6.53 s


In [18]:
%%time
# 3단계: 데이터 합치기
data_ac = pd.merge(activity, combat, on = ['acc_id','day'])
# 현금 결제만 하고 플레이는 안 한 경우가 있어서, 인스턴스가 945628 -> 946632로 증가 (결측값을 0으로 채움)
data_acp = pd.merge(data_ac, payment, on = ['acc_id','day'], how='outer').fillna(0) 
data_acpl = pd.merge(data_acp, pledge, on = ['acc_id','day'], how='outer').fillna(0)
data_acplt = pd.merge(data_acpl, trade, on = ['acc_id','day'], how='outer').dropna(
                    subset = ['playtime']).fillna(0)

Wall time: 50.2 s


In [19]:
data_acplt.to_csv('data_acplt.csv')

# Memory Error로 데이터 read

In [3]:
data_acplt = pd.read_csv('data_acplt.csv')

In [4]:
acc_id = np.sort(train_activity['acc_id'].unique())

In [7]:
data_acplt.head()

,acc_id,day,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,...,temp_cnt_y,etc_cnt_y,combat_play_time,non_combat_play_time,type_x,item_amount_s,item_price_s,type_y,item_amount_t,item_price_t
0,2,1,2.703457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.732562e-06,3.456524,0.0,0.0,0.0
1,2,2,3.363522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.116080e-01,0.201926,0.0,0.0,0.0
2,2,3,3.370544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.430650e-07,1.532037,0.0,0.0,0.0
3,2,4,3.370544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,0.0
4,2,5,3.370544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.396984e-08,0.075984,0.0,0.0,0.0


In [12]:
%time ind = pd.MultiIndex.from_product([acc_id.tolist(), list(range(1, 29))], names=['acc_id', 'day'])

Wall time: 173 ms


In [13]:
%%time
data = data_acplt.set_index(['acc_id', 'day']).reindex(ind)
data = data.fillna(0).reset_index()

Wall time: 4.86 s


In [14]:
data.head()

,acc_id,day,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,...,temp_cnt_y,etc_cnt_y,combat_play_time,non_combat_play_time,type_x,item_amount_s,item_price_s,type_y,item_amount_t,item_price_t
0,2,1,2.703457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.732562e-06,3.456524,0.0,0.0,0.0
1,2,2,3.363522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.116080e-01,0.201926,0.0,0.0,0.0
2,2,3,3.370544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.430650e-07,1.532037,0.0,0.0,0.0
3,2,4,3.370544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,0.0
4,2,5,3.370544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.396984e-08,0.075984,0.0,0.0,0.0


In [15]:
data.shape # 40000 * 28

(1120000, 42)

In [18]:
np.random.seed(42)
np.random.shuffle(acc_id)

In [27]:
extra, test_ind = acc_id[:int(.7 * len(acc_id))], acc_id[int(.7 * len(acc_id)):]
train_ind, valid_ind = extra[:int(.75 * len(extra))], extra[int(.75 * len(extra)):]

In [28]:
len(train_ind), len(valid_ind), len(test_ind)

(21000, 7000, 12000)

In [34]:
d = data.set_index('acc_id')

In [40]:
label = np.where(train_label.sort_values('acc_id')['survival_time'] == 64, 1, 0)
label = pd.Series(data=label, index=train_label.sort_values('acc_id')['acc_id'])

In [41]:
X_train = d.loc[train_ind].reset_index()
y_train = label.loc[train_ind].values

X_valid = d.loc[valid_ind].reset_index()
y_valid = label.loc[valid_ind].values

X_test = d.loc[test_ind].reset_index()
y_test = label.loc[test_ind].values

In [43]:
X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape

((588000, 42), (196000, 42), (336000, 42), (21000,), (7000,), (12000,))

In [60]:
# index masking
X_train['acc_id'] = np.tile(np.arange(int(X_train.shape[0] / 28)).reshape(-1, 1), 28).flatten()
X_valid['acc_id'] = np.tile(np.arange(int(X_valid.shape[0] / 28)).reshape(-1, 1), 28).flatten()
X_test['acc_id'] = np.tile(np.arange(int(X_test.shape[0] / 28)).reshape(-1, 1), 28).flatten()

# day -= 1
X_train['day'] = X_train['day'] - 1
X_valid['day'] = X_valid['day'] - 1
X_test['day'] = X_test['day'] - 1

In [70]:
X_train = X_train.values
X_valid = X_valid.values
X_test = X_test.values

In [91]:
400 % (40000 // 100)

0

In [97]:
X_train.shape

(588000, 42)

In [94]:
def build_data(acc_id, day, X, max_time, is_test, num_user=None, y=None):
    if num_user == None:
        num_user = np.unique(X_train[:, 0]).shape[0]
    # y[0] will be days remaining, y[1] will be event indicator, always 1 for this data
    out_y = np.empty((0, 2), dtype=np.float32)

    # A full history of sensor readings to date for each x
    out_x = np.empty((0, max_time, 24), dtype=np.float32)

    for i in range(num_user):
        if i % (num_user // 100) == 0:
            print("User: " + str(i))
        # When did the engine fail? (Last day + 1 for train data, irrelevant for test.)
        max_engine_time = int(np.max(day[acc_id == i])) + 1

        if is_test:
            start = max_engine_time - 1
        else:
            start = 0

        this_x = np.empty((0, max_time, 24), dtype=np.float32)

        for j in range(start, max_engine_time):
            engine_x = X[acc_id == i]

            out_y = np.append(out_y, np.array((max_engine_time - j, y[i]), ndmin=2), axis=0)

            xtemp = np.zeros((1, max_time, 24))
            xtemp[:, max_time-min(j, num_user-1)-1:max_time, :] = \
                        engine_x[max(0, j-max_time+1):j+1, :]
            this_x = np.concatenate((this_x, xtemp))

        out_x = np.concatenate((out_x, this_x))

    return out_x, out_y

In [95]:
max_time = 28

train_x, train_y = build_data(X_train[:, 0], X_train[:, 1], X_train[:, 2:26], 
                              max_time, False, y=y_train)
valid_x, valid_y = build_data(X_valid[:, 0], X_valid[:, 1], X_valid[:, 2:26], 
                              max_time, False, y=y_valid)
test_x, _ = build_data(X_test[:, 0], X_test[:, 1], X_test[:, 2:26], 
                       max_time, True)

User: 0
User: 210
User: 420
User: 630
User: 840
User: 1050
User: 1260


MemoryError: 

In [67]:
X_train

,acc_id,day,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,...,temp_cnt_y,etc_cnt_y,combat_play_time,non_combat_play_time,type_x,item_amount_s,item_price_s,type_y,item_amount_t,item_price_t
0,0,0,2.162765,0.169984,0.006535,0.000794,0.000000,0.0,0.000000,0.000000,...,0.395532,0.481322,1.033676,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000
1,0,1,3.377565,1.536276,0.009131,0.002227,0.000217,1.0,0.000000,0.000000,...,0.000000,0.920174,1.257462,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000
2,0,2,3.377565,2.182079,0.011422,0.000675,0.000651,1.0,0.000000,0.000000,...,1.384363,0.184035,0.946803,0.000000,0.0,0.000000e+00,0.000000,0.0,2.396984e-08,-0.062140
3,0,3,3.377565,1.356830,0.007960,0.000000,0.000434,0.0,0.000000,0.000000,...,0.000000,0.042470,0.988502,0.000000,1.0,4.793968e-02,0.000000,0.0,2.396984e-08,-0.507610
4,0,4,3.382247,3.548709,0.018367,0.000000,0.001518,0.0,0.000000,0.000000,...,0.197766,0.127409,1.159469,0.000000,1.0,5.417184e-06,0.000000,2.0,4.799097e-03,0.000000
5,0,5,3.375225,1.796152,0.009215,0.000000,0.000651,0.0,0.000000,0.000000,...,0.395532,0.169878,1.118928,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000
6,0,6,3.379906,0.364637,0.004522,0.001159,0.000000,0.0,0.000000,0.000000,...,1.186597,0.127409,1.361478,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000
7,0,7,2.415556,1.513972,0.010106,0.000000,0.000651,1.0,0.000000,0.000000,...,0.593298,0.495479,1.140936,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000
8,0,8,3.382247,0.007773,0.002846,0.000812,0.000217,1.0,0.000000,0.000000,...,0.197766,0.353913,0.894216,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000
9,0,9,3.382247,0.006083,0.005969,0.000937,0.000217,1.0,0.000000,0.000000,...,0.395532,0.212348,0.958618,0.000000,0.0,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000
